In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [4]:
!pip install --upgrade datasets

In [6]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=122147fd3184cfc6dcfa24b6bbfc1075bdb0c8755b8d6db11b00d05008048907
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
# -*- coding: utf-8 -*-
"""
Enhanced GPU-Optimized Self-Adaptive GPT2 with GRPO Training + CEM Inference
Updated with robust HuggingFace dataset integration and improved dataset variety
"""

import subprocess
import sys
import warnings
import logging
import traceback
import time
import requests
from contextlib import contextmanager
warnings.filterwarnings("ignore")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

def install_packages():
    """Enhanced package installation with better error handling"""
    packages = [
        "torch>=1.9.0", "transformers>=4.20.0", "datasets>=2.0.0", "wandb",
        "numpy>=1.21.0", "scipy>=1.7.0", "matplotlib>=3.3.0", "seaborn>=0.11.0",
        "accelerate>=0.12.0", "evaluate>=0.2.0", "rouge-score>=0.1.0",
        "sacrebleu>=2.0.0", "bert-score>=0.3.0", "scikit-learn>=1.0.0",
        "pandas>=1.3.0", "tqdm>=4.60.0", "sentence-transformers>=2.0.0",
        "requests>=2.25.0", "tokenizers>=0.12.0", "huggingface-hub>=0.10.0"
    ]

    success_count = 0
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
            logger.info(f"✓ Successfully installed {package}")
            success_count += 1
        except subprocess.CalledProcessError as e:
            logger.warning(f"✗ Failed to install {package}: {str(e)}")

    logger.info(f"Package installation complete: {success_count}/{len(packages)} successful")
    return success_count > len(packages) * 0.8  # 80% success rate

# Uncomment on first run
# install_packages()

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
import os
import gc
import math
import random
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Any, Union
from collections import defaultdict, deque
from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer,
    get_cosine_schedule_with_warmup,
    AutoTokenizer, AutoModel
)
from datasets import load_dataset, concatenate_datasets, Dataset
import wandb
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import evaluate
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util

@contextmanager
def timer(description: str):
    """Context manager for timing operations"""
    start = time.time()
    yield
    elapsed = time.time() - start
    logger.info(f"{description}: {elapsed:.2f}s")

def setup_device():
    try:
        if torch.cuda.is_available():
            device = torch.device("cuda")
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            logger.info(f"Using GPU: {torch.cuda.get_device_name()}")
            logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
            logger.info(f"CUDA Version: {torch.version.cuda}")
        else:
            device = torch.device("cpu")
            logger.warning("Using CPU - training will be slower")
        return device
    except Exception as e:
        logger.error(f"Device setup failed: {str(e)}")
        return torch.device("cpu")

device = setup_device()

@dataclass
class EnhancedConfig:
    # Model configuration
    model_name: str = "gpt2"
    batch_size: int = 16
    learning_rate: float = 5e-5
    num_epochs: int = 5
    max_length: int = 256
    adaptation_rank: int = 32
    num_experts: int = 8

    # Training optimization
    mixed_precision: bool = False
    gradient_accumulation_steps: int = 4
    max_grad_norm: float = 0.5
    warmup_steps: int = 100
    weight_decay: float = 0.01

    # Enhanced dataset configuration
    max_samples_per_dataset: int = 500
    use_fallback_data_only: bool = False
    enable_internet_check: bool = True
    dataset_download_timeout: int = 300
    max_download_retries: int = 3

    # Dataset variety controls
    enable_qa_datasets: bool = True
    enable_sentiment_datasets: bool = True
    enable_summarization_datasets: bool = True
    enable_classification_datasets: bool = True
    enable_generation_datasets: bool = True

    # GRPO parameters
    grpo_episodes_per_batch: int = 8
    grpo_reward_normalization: bool = True
    grpo_kl_coeff: float = 0.01
    grpo_value_loss_coeff: float = 0.1
    grpo_entropy_coeff: float = 0.08

    # CEM parameters
    cem_population_size: int = 100
    cem_elite_ratio: float = 0.3
    cem_noise_std: float = 0.3
    cem_adaptation_steps: int = 50
    cem_convergence_threshold: float = 5e-3
    cem_momentum: float = 0.3

    # SVD parameters
    svd_rank_ratio: float = 0.8
    svd_min_singular_value: float = 1e-5

    # Logging and saving
    wandb_project: str = "enhanced-grpo-cem-gpt2"
    output_dir: str = "./enhanced_results"
    log_interval: int = 10
    save_interval: int = 1

    # Stability improvements
    clip_rewards: float = 3.0
    reward_scaling: float = 0.1
    temperature_annealing: bool = True
    adaptive_learning_rate: bool = True

    # Generation parameters
    repetition_penalty: float = 1.3
    top_p: float = 0.85
    temperature: float = 0.6

class StabilizedSVDDecomposer:
    @staticmethod
    def decompose_weight(weight: torch.Tensor, rank_ratio: float = 0.8, min_sv: float = 1e-5):
        try:
            weight = weight.to(device).float()
            reg_weight = weight + torch.randn_like(weight) * 1e-8
            U, S, Vh = torch.linalg.svd(reg_weight, full_matrices=False)
            V = Vh.T

            valid_sv = S > min_sv
            if rank_ratio < 1.0:
                n_keep = max(1, int(len(S) * rank_ratio))
                keep_indices = torch.argsort(S, descending=True)[:n_keep]
                keep_mask = torch.zeros_like(S, dtype=torch.bool, device=device)
                keep_mask[keep_indices] = True
                valid_sv = valid_sv & keep_mask

            return U[:, valid_sv], S[valid_sv], V[:, valid_sv]
        except Exception as e:
            logger.error(f"SVD decomposition failed: {str(e)}")
            return None, None, None

    @staticmethod
    def reconstruct_weight(U: torch.Tensor, S: torch.Tensor, V: torch.Tensor,
                         adaptation_vector: torch.Tensor = None):
        if any(x is None for x in [U, S, V]):
            return None
        try:
            if adaptation_vector is not None:
                adaptation_factor = torch.tanh(adaptation_vector[:len(S)]) * 0.1 + 1.0
                adapted_S = S * adaptation_factor
                scale_factor = S.sum() / (adapted_S.sum() + 1e-8)
                adapted_S = adapted_S * scale_factor
            else:
                adapted_S = S
            S_diag = torch.diag(adapted_S)
            return torch.chain_matmul(U, S_diag, V.T)
        except Exception as e:
            logger.error(f"Weight reconstruction failed: {str(e)}")
            return None

class EnhancedValueNetwork(nn.Module):
    def __init__(self, hidden_size: int, dropout: float = 0.1):
        super().__init__()
        self.value_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LayerNorm(hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.LayerNorm(hidden_size // 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 4, 1)
        ).to(device)

        for module in self.value_head:
            if isinstance(module, nn.Linear):
                nn.init.orthogonal_(module.weight, gain=np.sqrt(2))
                nn.init.zeros_(module.bias)

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        try:
            hidden_states = hidden_states.to(device)
            attention_weights = F.softmax(
                torch.mean(hidden_states, dim=-1), dim=-1
            ).unsqueeze(-1)
            pooled = torch.sum(hidden_states * attention_weights, dim=1)
            return self.value_head(pooled).squeeze(-1)
        except Exception as e:
            logger.error(f"Value network forward failed: {str(e)}")
            return torch.zeros(hidden_states.size(0), device=device)

class ImprovedTaskRewardFunction:
    def __init__(self):
        self.rouge = evaluate.load("rouge")
        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        self.task_scales = {
            'qa': 3.0,
            'summarization': 2.5,
            'sentiment': 2.0,
            'classification': 2.2,
            'general': 1.5
        }
        logger.info("Initialized reward function with enhanced scaling")

    def compute_reward(self, generated_text: str, target_text: str, task_type: str) -> float:
        try:
            if not generated_text or not target_text:
                return -2.0

            reward = 0.0
            if task_type == "qa":
                reward = self._qa_reward(generated_text, target_text)
            elif task_type == "summarization":
                reward = self._summarization_reward(generated_text, target_text)
            elif task_type == "sentiment":
                reward = self._sentiment_reward(generated_text, target_text)
            elif task_type == "classification":
                reward = self._classification_reward(generated_text, target_text)
            else:
                reward = self._general_reward(generated_text, target_text)

            scaled_reward = reward * self.task_scales.get(task_type, 1.0)
            return np.clip(scaled_reward, -3.0, 3.0)
        except Exception as e:
            logger.error(f"Reward computation failed for {task_type}: {str(e)}")
            return -2.0

    def _qa_reward(self, generated: str, target: str) -> float:
        generated_lower = generated.lower().strip()
        target_lower = target.lower().strip()

        # Exact match bonus
        if generated_lower == target_lower:
            return 2.0

        # Containment bonus
        if target_lower in generated_lower:
            return 1.5

        # Word overlap scoring
        gen_words = generated_lower.split()
        target_words = target_lower.split()
        if not target_words:
            return 0.0

        overlap_score = sum(1.0 / (i + 1) for i, word in enumerate(target_words) if word in gen_words)
        overlap_score /= sum(1.0 / (i + 1) for i in range(len(target_words)))

        # Semantic similarity
        embeddings = self.sentence_encoder.encode([generated, target], convert_to_tensor=True)
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()

        # Length penalty
        length_penalty = min(len(generated.split()) / 50, 1.0)

        # Factuality bonus
        factuality_score = 1.0 if target_lower in generated_lower else 0.5

        return (overlap_score * 0.4 + similarity * 0.4 + factuality_score * 0.2) * length_penalty

    def _summarization_reward(self, generated: str, target: str) -> float:
        gen_len = len(generated.split())
        target_len = len(target.split())

        if gen_len == 0 or target_len == 0:
            return -1.0

        # Length ratio scoring
        length_ratio = min(gen_len / target_len, target_len / gen_len)
        length_score = 1.0 if 0.5 <= length_ratio <= 1.5 else 0.5

        # ROUGE scoring
        rouge_scores = self.rouge.compute(predictions=[generated], references=[target])
        rouge_avg = (rouge_scores['rouge1'] + rouge_scores['rouge2'] + rouge_scores['rougeL']) / 3

        # Semantic similarity
        embeddings = self.sentence_encoder.encode([generated, target], convert_to_tensor=True)
        semantic_similarity = util.cos_sim(embeddings[0], embeddings[1]).item()

        return (rouge_avg * 0.5 + semantic_similarity * 0.3 + length_score * 0.2) * 1.2

    def _sentiment_reward(self, generated: str, target: str) -> float:
        positive_words = {
            'good', 'great', 'excellent', 'positive', 'happy', 'love', 'amazing',
            'wonderful', 'fantastic', 'awesome', 'brilliant', 'perfect'
        }
        negative_words = {
            'bad', 'terrible', 'awful', 'negative', 'sad', 'hate', 'horrible',
            'disgusting', 'worst', 'disappointing', 'annoying', 'frustrating'
        }

        gen_words = set(generated.lower().split())
        target_lower = target.lower()

        gen_positive = len(gen_words & positive_words)
        gen_negative = len(gen_words & negative_words)

        target_is_positive = any(word in target_lower for word in ['positive', '1', 'good', 'great'])

        confidence_score = abs(gen_positive - gen_negative) / max(gen_positive + gen_negative, 1)

        if target_is_positive and gen_positive > gen_negative:
            return 1.5 * confidence_score
        elif not target_is_positive and gen_negative > gen_positive:
            return 1.5 * confidence_score
        elif gen_positive == gen_negative:
            return 0.8 * confidence_score
        else:
            return 0.2 * confidence_score

    def _classification_reward(self, generated: str, target: str) -> float:
        """Reward function for classification tasks"""
        generated_lower = generated.lower().strip()
        target_lower = target.lower().strip()

        # Exact match
        if generated_lower == target_lower:
            return 2.0

        # Partial match (target in generated)
        if target_lower in generated_lower:
            return 1.5

        # Check for synonyms or related terms
        category_synonyms = {
            'world': ['global', 'international', 'politics', 'nation'],
            'sports': ['athletics', 'games', 'competition', 'team'],
            'business': ['finance', 'economy', 'market', 'company'],
            'technology': ['tech', 'computer', 'digital', 'software']
        }

        for category, synonyms in category_synonyms.items():
            if category in target_lower:
                if any(syn in generated_lower for syn in synonyms):
                    return 1.2

        # Semantic similarity
        embeddings = self.sentence_encoder.encode([generated, target], convert_to_tensor=True)
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()

        return similarity * 0.8

    def _general_reward(self, generated: str, target: str) -> float:
        if len(generated.strip()) < 5:
            return -1.0

        words = generated.split()
        if not words:
            return -1.0

        # Diversity score
        unique_words = len(set(words))
        diversity = unique_words / len(words)

        # Length score
        length_score = min(len(words) / 50, 1.0)

        # Fluency score
        sentences = [s.strip() for s in generated.split('.') if s.strip()]
        fluency_score = min(len(sentences) / 3, 1.0) * 0.3

        # Coherence score
        embeddings = self.sentence_encoder.encode([generated, target], convert_to_tensor=True)
        coherence_score = util.cos_sim(embeddings[0], embeddings[1]).item()

        return (diversity * 0.3 + length_score * 0.3 + fluency_score * 0.2 + coherence_score * 0.2) * 1.2

class RobustDatasetLoader:
    def __init__(self, config: EnhancedConfig):
        self.config = config
        self.datasets = {}
        self.validation_datasets = {}
        self.successful_downloads = 0
        self.failed_downloads = 0

    def check_internet_connection(self):
        """Check if we can reach Hugging Face Hub"""
        if not self.config.enable_internet_check:
            return True
        try:
            response = requests.get("https://huggingface.co", timeout=10)
            return response.status_code == 200
        except:
            return False

    def download_with_retry(self, dataset_name, subset=None, split='train', max_retries=None):
        """Download dataset with retry logic and better error handling"""
        if max_retries is None:
            max_retries = self.config.max_download_retries

        for attempt in range(max_retries):
            try:
                logger.info(f"Downloading {dataset_name} (attempt {attempt + 1}/{max_retries})")

                if subset:
                    dataset = load_dataset(
                        dataset_name,
                        subset,
                        split=split,
                        download_mode="reuse_cache_if_exists",
                        verification_mode="no_checks"
                    )
                else:
                    dataset = load_dataset(
                        dataset_name,
                        split=split,
                        download_mode="reuse_cache_if_exists",
                        verification_mode="no_checks"
                    )

                logger.info(f"Successfully loaded {dataset_name} with {len(dataset)} samples")
                return dataset

            except Exception as e:
                logger.warning(f"Attempt {attempt + 1} failed for {dataset_name}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                continue

        logger.error(f"Failed to load {dataset_name} after {max_retries} attempts")
        return None

    def load_all_datasets(self):
        """Load datasets with improved error handling and more variety"""

        if self.config.use_fallback_data_only:
            logger.info("Using fallback data only (as specified in config)")
            self._add_comprehensive_fallback_data()
            return self.datasets

        # Check internet connection first
        if not self.check_internet_connection():
            logger.warning("No internet connection detected, using fallback data")
            self._add_comprehensive_fallback_data()
            return self.datasets

        # Enhanced dataset configurations
        dataset_configs = []

        # Question Answering datasets
        if self.config.enable_qa_datasets:
            dataset_configs.extend([
                {
                    'name': 'squad',
                    'subset': None,
                    'split': 'train[:1000]',
                    'val_split': 'validation[:200]',
                    'task_type': 'qa',
                    'process_fn': self._process_squad,
                    'max_samples': 400,
                    'priority': 1
                },
                {
                    'name': 'microsoft/ms_marco',
                    'subset': 'v1.1',
                    'split': 'train[:800]',
                    'val_split': 'validation[:150]',
                    'task_type': 'qa',
                    'process_fn': self._process_ms_marco,
                    'max_samples': 300,
                    'priority': 2
                }
            ])

        # Sentiment Analysis datasets
        if self.config.enable_sentiment_datasets:
            dataset_configs.extend([
                {
                    'name': 'imdb',
                    'subset': None,
                    'split': 'train[:1000]',
                    'val_split': 'test[:200]',
                    'task_type': 'sentiment',
                    'process_fn': self._process_imdb,
                    'max_samples': 400,
                    'priority': 1
                },
                {
                    'name': 'amazon_polarity',
                    'subset': None,
                    'split': 'train[:800]',
                    'val_split': 'test[:150]',
                    'task_type': 'sentiment',
                    'process_fn': self._process_amazon_polarity,
                    'max_samples': 300,
                    'priority': 2
                },
                {
                    'name': 'yelp_review_full',
                    'subset': None,
                    'split': 'train[:600]',
                    'val_split': 'test[:100]',
                    'task_type': 'sentiment',
                    'process_fn': self._process_yelp_reviews,
                    'max_samples': 250,
                    'priority': 3
                }
            ])

        # Summarization datasets
        if self.config.enable_summarization_datasets:
            dataset_configs.extend([
                {
                    'name': 'xsum',
                    'subset': None,
                    'split': 'train[:600]',
                    'val_split': 'validation[:100]',
                    'task_type': 'summarization',
                    'process_fn': self._process_xsum,
                    'max_samples': 250,
                    'priority': 1
                },
                {
                    'name': 'cnn_dailymail',
                    'subset': '3.0.0',
                    'split': 'train[:500]',
                    'val_split': 'validation[:80]',
                    'task_type': 'summarization',
                    'process_fn': self._process_cnn_dailymail,
                    'max_samples': 200,
                    'priority': 2
                }
            ])

        # Classification datasets
        if self.config.enable_classification_datasets:
            dataset_configs.extend([
                {
                    'name': 'ag_news',
                    'subset': None,
                    'split': 'train[:600]',
                    'val_split': 'test[:100]',
                    'task_type': 'classification',
                    'process_fn': self._process_ag_news,
                    'max_samples': 250,
                    'priority': 2
                },
                {
                    'name': 'SetFit/20_newsgroups',
                    'subset': None,
                    'split': 'train[:500]',
                    'val_split': 'test[:80]',
                    'task_type': 'classification',
                    'process_fn': self._process_newsgroups,
                    'max_samples': 200,
                    'priority': 3
                }
            ])

        # Text generation datasets
        if self.config.enable_generation_datasets:
            dataset_configs.extend([
                {
                    'name': 'wikitext',
                    'subset': 'wikitext-2-raw-v1',
                    'split': 'train[:800]',
                    'val_split': 'validation[:100]',
                    'task_type': 'general',
                    'process_fn': self._process_wikitext,
                    'max_samples': 300,
                    'priority': 1
                },
                {
                    'name': 'roneneldan/TinyStories',
                    'subset': None,
                    'split': 'train[:600]',
                    'val_split': 'validation[:80]',
                    'task_type': 'general',
                    'process_fn': self._process_tiny_stories,
                    'max_samples': 250,
                    'priority': 2
                }
            ])

        # Sort by priority
        dataset_configs.sort(key=lambda x: x['priority'])

        logger.info(f"Attempting to load {len(dataset_configs)} HuggingFace datasets")

        for config in dataset_configs:
            try:
                logger.info(f"Loading {config['name']} for {config['task_type']} task")

                # Download training data
                dataset = self.download_with_retry(
                    config['name'],
                    config['subset'],
                    config['split']
                )

                if dataset is None:
                    self.failed_downloads += 1
                    logger.warning(f"Skipping {config['name']} - download failed")
                    continue

                # Download validation data if available
                val_dataset = None
                if config['val_split']:
                    val_dataset = self.download_with_retry(
                        config['name'],
                        config['subset'],
                        config['val_split']
                    )

                # Process the datasets
                processed_data = config['process_fn'](dataset)
                val_processed = config['process_fn'](val_dataset) if val_dataset else []

                if processed_data and len(processed_data) > 0:
                    task_type = config['task_type']
                    if task_type not in self.datasets:
                        self.datasets[task_type] = []
                        self.validation_datasets[task_type] = []

                    # Limit samples as specified
                    max_samples = min(len(processed_data), config['max_samples'])
                    selected_data = processed_data[:max_samples]

                    self.datasets[task_type].extend(selected_data)
                    self.validation_datasets[task_type].extend(val_processed[:50])

                    self.successful_downloads += 1
                    logger.info(f"✓ Added {len(selected_data)} training samples and "
                              f"{len(val_processed[:50])} validation samples from {config['name']}")
                else:
                    logger.warning(f"No valid samples extracted from {config['name']}")

            except Exception as e:
                self.failed_downloads += 1
                logger.error(f"Failed to process {config['name']}: {str(e)}")
                continue

        # Summary
        total_samples = sum(len(data) for data in self.datasets.values())
        logger.info(f"\nDataset Loading Summary:")
        logger.info(f"✓ Successful downloads: {self.successful_downloads}")
        logger.info(f"✗ Failed downloads: {self.failed_downloads}")
        logger.info(f"📊 Total training samples: {total_samples:,}")

        # If we didn't get enough data, add supplementary fallback
        if self.successful_downloads < 2 or total_samples < 500:
            logger.warning("Insufficient real data loaded, adding supplementary fallback data")
            self._add_supplementary_fallback_data()

        # Log final statistics
        for task, data in self.datasets.items():
            val_count = len(self.validation_datasets.get(task, []))
            logger.info(f"  📋 {task}: {len(data)} training + {val_count} validation samples")

        return self.datasets

    # Dataset processing methods
    def _process_squad(self, dataset):
        """Process SQuAD dataset"""
        processed = []
        for item in dataset:
            try:
                context = item.get('context', '').strip()
                question = item.get('question', '').strip()
                answers = item.get('answers', {})

                if not context or not question or not answers or not answers.get('text'):
                    continue

                answer = answers['text'][0].strip()
                if len(answer) > 0:
                    context_truncated = context[:300] + "..." if len(context) > 300 else context
                    input_text = f"Context: {context_truncated}\nQuestion: {question}"
                    processed.append((input_text, answer, 'qa'))
            except Exception:
                continue
        return processed

    def _process_ms_marco(self, dataset):
        """Process MS MARCO dataset"""
        processed = []
        for item in dataset:
            try:
                query = item.get('query', '').strip()
                passages = item.get('passages', [])
                answers = item.get('answers', [])

                if not query or not passages or not answers:
                    continue

                # Use the first passage as context
                context = passages[0].get('passage_text', '').strip() if passages else ''
                answer = answers[0].strip() if answers else ''

                if context and answer and len(answer) > 0:
                    context_truncated = context[:200] + "..." if len(context) > 200 else context
                    input_text = f"Context: {context_truncated}\nQuestion: {query}"
                    processed.append((input_text, answer, 'qa'))
            except Exception:
                continue
        return processed

    def _process_imdb(self, dataset):
        """Process IMDB dataset"""
        processed = []
        for item in dataset:
            try:
                text = item.get('text', '').strip()
                label = item.get('label', 0)

                if len(text) > 50:
                    text = text[:400] + "..." if len(text) > 400 else text
                    target = 'positive' if label == 1 else 'negative'
                    input_text = f"Analyze the sentiment of this review: {text}"
                    processed.append((input_text, target, 'sentiment'))
            except Exception:
                continue
        return processed

    def _process_amazon_polarity(self, dataset):
        """Process Amazon Polarity dataset"""
        processed = []
        for item in dataset:
            try:
                title = item.get('title', '').strip()
                content = item.get('content', '').strip()
                label = item.get('label', 0)

                text = f"{title}. {content}".strip()
                if len(text) > 50:
                    text = text[:350] + "..." if len(text) > 350 else text
                    target = 'positive' if label == 1 else 'negative'
                    input_text = f"What is the sentiment of this Amazon review: {text}"
                    processed.append((input_text, target, 'sentiment'))
            except Exception:
                continue
        return processed

    def _process_yelp_reviews(self, dataset):
        """Process Yelp Reviews dataset"""
        processed = []
        for item in dataset:
            try:
                text = item.get('text', '').strip()
                label = item.get('label', 0)  # 0-4 scale

                if len(text) > 50:
                    text = text[:350] + "..." if len(text) > 350 else text
                    # Convert 5-scale to binary sentiment
                    target = 'positive' if label >= 3 else 'negative'
                    input_text = f"Determine sentiment of this Yelp review: {text}"
                    processed.append((input_text, target, 'sentiment'))
            except Exception:
                continue
        return processed

    def _process_xsum(self, dataset):
        """Process XSum dataset"""
        processed = []
        for item in dataset:
            try:
                document = item.get('document', '').strip()
                summary = item.get('summary', '').strip()

                if len(document) > 100 and len(summary) > 10:
                    doc_truncated = document[:500] + "..." if len(document) > 500 else document
                    input_text = f"Summarize this article: {doc_truncated}"
                    processed.append((input_text, summary, 'summarization'))
            except Exception:
                continue
        return processed

    def _process_cnn_dailymail(self, dataset):
        """Process CNN/DailyMail dataset"""
        processed = []
        for item in dataset:
            try:
                article = item.get('article', '').strip()
                highlights = item.get('highlights', '').strip()

                if len(article) > 150 and len(highlights) > 15:
                    article_truncated = article[:600] + "..." if len(article) > 600 else article
                    input_text = f"Summarize this news article: {article_truncated}"
                    processed.append((input_text, highlights, 'summarization'))
            except Exception:
                continue
        return processed

    def _process_ag_news(self, dataset):
        """Process AG News dataset"""
        processed = []
        label_map = {0: 'world', 1: 'sports', 2: 'business', 3: 'technology'}

        for item in dataset:
            try:
                text = item.get('text', '').strip()
                label = item.get('label', 0)

                if len(text) > 30:
                    text = text[:300] + "..." if len(text) > 300 else text
                    target = label_map.get(label, 'general')
                    input_text = f"Classify this news article: {text}"
                    processed.append((input_text, target, 'classification'))
            except Exception:
                continue
        return processed

    def _process_newsgroups(self, dataset):
        """Process 20 Newsgroups dataset"""
        processed = []
        for item in dataset:
            try:
                text = item.get('text', '').strip()
                label_text = item.get('label_text', '').strip()

                if len(text) > 50 and label_text:
                    text = text[:400] + "..." if len(text) > 400 else text
                    # Simplify newsgroup categories
                    simplified_label = self._simplify_newsgroup_label(label_text)
                    input_text = f"What category does this text belong to: {text}"
                    processed.append((input_text, simplified_label, 'classification'))
            except Exception:
                continue
        return processed

    def _simplify_newsgroup_label(self, label):
        """Simplify newsgroup labels to broader categories"""
        label_lower = label.lower()
        if any(word in label_lower for word in ['comp', 'computer', 'tech']):
            return 'technology'
        elif any(word in label_lower for word in ['sci', 'science', 'space']):
            return 'science'
        elif any(word in label_lower for word in ['rec', 'sport', 'auto']):
            return 'recreation'
        elif any(word in label_lower for word in ['talk', 'soc', 'politics']):
            return 'discussion'
        else:
            return 'general'

    def _process_wikitext(self, dataset):
        """Process WikiText dataset"""
        processed = []
        for item in dataset:
            try:
                text = item.get('text', '').strip()

                # Skip empty lines and headers
                if len(text) < 100 or text.startswith('=') or not text:
                    continue

                sentences = text.split('.')
                if len(sentences) >= 2:
                    input_part = sentences[0].strip()
                    target_part = '.'.join(sentences[1:3]).strip()  # Use 2 sentences as target

                    if len(input_part) > 20 and len(target_part) > 30:
                        input_text = f"Continue this text: {input_part}."
                        processed.append((input_text, target_part, 'general'))
            except Exception:
                continue
        return processed

    def _process_tiny_stories(self, dataset):
        """Process TinyStories dataset"""
        processed = []
        for item in dataset:
            try:
                text = item.get('text', '').strip()

                if len(text) > 100:
                    sentences = text.split('.')
                    if len(sentences) >= 3:
                        input_part = '.'.join(sentences[:2]).strip()
                        target_part = '.'.join(sentences[2:4]).strip()

                        if len(input_part) > 30 and len(target_part) > 20:
                            input_text = f"Continue this story: {input_part}."
                            processed.append((input_text, target_part, 'general'))
            except Exception:
                continue
        return processed

    def _add_supplementary_fallback_data(self):
        """Add high-quality fallback data to supplement downloaded datasets"""
        logger.info("Adding supplementary fallback data")

        supplementary_data = {
            'qa': [
                ("What is the capital of France?", "Paris", "qa"),
                ("Who wrote Romeo and Juliet?", "William Shakespeare", "qa"),
                ("What is the largest planet in our solar system?", "Jupiter", "qa"),
                ("What year did World War II end?", "1945", "qa"),
                ("What is the chemical symbol for gold?", "Au", "qa"),
                ("Who painted the Mona Lisa?", "Leonardo da Vinci", "qa"),
                ("What is the speed of light?", "299,792,458 meters per second", "qa"),
                ("What is the smallest country in the world?", "Vatican City", "qa"),
                ("What is the hardest natural substance?", "Diamond", "qa"),
                ("What gas do plants absorb during photosynthesis?", "Carbon dioxide", "qa"),
            ],
            'sentiment': [
                ("This movie is absolutely amazing! The acting was superb and the plot was engaging.", "positive", "sentiment"),
                ("I hate this product. It's completely useless and a waste of money.", "negative", "sentiment"),
                ("The service at this restaurant was excellent. Highly recommend!", "positive", "sentiment"),
                ("This software is terrible. It crashes constantly and has poor design.", "negative", "sentiment"),
                ("I love this book! It's well-written and captivating from start to finish.", "positive", "sentiment"),
                ("The weather today is beautiful and perfect for outdoor activities.", "positive", "sentiment"),
                ("This device is frustrating to use. The interface is confusing and slow.", "negative", "sentiment"),
                ("Outstanding customer support! They solved my problem quickly and professionally.", "positive", "sentiment"),
                ("Worst experience ever. The staff was rude and unhelpful.", "negative", "sentiment"),
                ("Fantastic quality and great value for money. Very satisfied with this purchase.", "positive", "sentiment"),
            ],
            'summarization': [
                ("Climate change is affecting global weather patterns, causing more frequent extreme weather events, rising sea levels, and shifts in precipitation patterns. Scientists warn that immediate action is needed to reduce greenhouse gas emissions and transition to renewable energy sources.", "Climate change causes extreme weather and rising seas, requiring immediate emission reductions.", "summarization"),
                ("Artificial intelligence technology is rapidly advancing across multiple industries, from healthcare and finance to transportation and entertainment. Machine learning algorithms are becoming more sophisticated, enabling computers to perform complex tasks that previously required human intelligence.", "AI is rapidly advancing across industries with sophisticated machine learning enabling complex automated tasks.", "summarization"),
                ("The global economy shows mixed signals with some regions experiencing growth while others face recession. Inflation rates vary significantly between countries, and supply chain disruptions continue to impact international trade and manufacturing.", "Global economy shows mixed signals with varying growth, inflation, and ongoing supply chain disruptions.", "summarization"),
                ("Space exploration has reached new milestones with successful missions to Mars, lunar exploration programs, and the development of commercial spaceflight. These achievements advance our understanding of the universe and drive technological innovation.", "Space exploration achieves Mars missions, lunar programs, and commercial flight, advancing knowledge and technology.", "summarization"),
                ("Digital transformation in education has accelerated, with online learning platforms, virtual classrooms, and AI-powered tutoring systems becoming mainstream. This shift is reshaping how students learn and teachers deliver content.", "Digital education transformation includes online platforms, virtual classrooms, and AI tutoring reshaping learning.", "summarization"),
            ],
            'classification': [
                ("Scientists have discovered a new species of deep-sea fish in the Pacific Ocean. The research team used advanced underwater robots to explore previously uncharted areas of the ocean floor.", "science", "classification"),
                ("The championship game ended with a dramatic overtime victory as the home team scored the winning goal in the final minutes of play.", "sports", "classification"),
                ("Tech company stocks surged after announcing breakthrough developments in quantum computing technology, with investors showing strong confidence in future market potential.", "technology", "classification"),
                ("The central bank announced a change in interest rates to combat inflation, affecting mortgage rates and consumer spending across the economy.", "business", "classification"),
                ("International leaders gathered at the summit to discuss global trade policies and diplomatic relations between major world powers.", "politics", "classification"),
                ("New archaeological findings suggest ancient civilizations had more advanced mathematical knowledge than previously thought.", "science", "classification"),
                ("The startup secured major funding to develop sustainable energy solutions for urban environments and smart city infrastructure.", "technology", "classification"),
                ("Market analysts predict significant changes in commodity prices due to seasonal demand fluctuations and supply chain adjustments.", "business", "classification"),
            ],
            'general': [
                ("The future of renewable energy looks promising", "Solar, wind, and hydroelectric power are becoming more efficient and cost-effective, making them viable alternatives to fossil fuels.", "general"),
                ("Space exploration continues to advance", "New technologies enable deeper space missions, satellite deployments, and potential human colonization of other planets.", "general"),
                ("Artificial intelligence is transforming industries", "Machine learning algorithms automate processes, improve decision-making, and create new possibilities in healthcare, finance, and transportation.", "general"),
                ("Climate change requires global cooperation", "International efforts to reduce emissions, develop clean technology, and adapt to environmental changes are essential for sustainability.", "general"),
                ("Education systems are adapting to digital learning", "Online platforms, interactive tools, and personalized learning experiences are reshaping how students acquire knowledge and skills.", "general"),
            ]
        }

        for task_type, data in supplementary_data.items():
            if task_type not in self.datasets:
                self.datasets[task_type] = []
                self.validation_datasets[task_type] = []

            # Add training data
            existing_count = len(self.datasets[task_type])
            self.datasets[task_type].extend(data)

            # Add validation data (subset)
            self.validation_datasets[task_type].extend(data[:3])

            logger.info(f"Added {len(data)} supplementary samples for {task_type} (was {existing_count})")

    def _add_comprehensive_fallback_data(self):
        """Add comprehensive fallback data when no internet or HuggingFace datasets available"""
        logger.info("Adding comprehensive fallback dataset")

        self.datasets = {
            'qa': [
                ("What is the capital of France?", "Paris", "qa"),
                ("Who wrote Romeo and Juliet?", "William Shakespeare", "qa"),
                ("What is 2 + 2?", "4", "qa"),
                ("What color is the sky?", "Blue", "qa"),
                ("What is the largest planet?", "Jupiter", "qa"),
                ("What is the smallest country?", "Vatican City", "qa"),
                ("Who painted the Mona Lisa?", "Leonardo da Vinci", "qa"),
                ("What is the speed of light?", "299,792,458 meters per second", "qa"),
                ("What is the chemical symbol for gold?", "Au", "qa"),
                ("What year did World War II end?", "1945", "qa"),
                ("What is the hardest natural substance?", "Diamond", "qa"),
                ("What gas do plants absorb?", "Carbon dioxide", "qa"),
                ("What is the freezing point of water?", "0 degrees Celsius", "qa"),
                ("Who invented the telephone?", "Alexander Graham Bell", "qa"),
                ("What is the largest ocean?", "Pacific Ocean", "qa"),
            ],
            'sentiment': [
                ("This movie is amazing!", "positive", "sentiment"),
                ("I hate this product.", "negative", "sentiment"),
                ("The service was excellent.", "positive", "sentiment"),
                ("This is terrible quality.", "negative", "sentiment"),
                ("I love this restaurant!", "positive", "sentiment"),
                ("The weather is beautiful today.", "positive", "sentiment"),
                ("This software is buggy and slow.", "negative", "sentiment"),
                ("Outstanding customer support!", "positive", "sentiment"),
                ("Worst experience ever.", "negative", "sentiment"),
                ("Absolutely fantastic work!", "positive", "sentiment"),
                ("This device is frustrating to use.", "negative", "sentiment"),
                ("Great value for money!", "positive", "sentiment"),
                ("Poor build quality and design.", "negative", "sentiment"),
                ("Exceeded my expectations completely.", "positive", "sentiment"),
                ("Disappointed with the results.", "negative", "sentiment"),
            ],
            'summarization': [
                ("Climate change is affecting global weather patterns and causing environmental issues. Rising temperatures lead to melting ice caps and rising sea levels.", "Climate change affects weather, causes ice melting and sea level rise.", "summarization"),
                ("Technology has revolutionized how we communicate and work in modern society. Smartphones and internet connectivity have changed everything.", "Technology changed communication and work through smartphones and internet.", "summarization"),
                ("The economy shows signs of recovery with increased employment and consumer spending. Market indicators suggest positive growth.", "Economy recovering with more jobs, spending, and positive growth.", "summarization"),
                ("Artificial intelligence is transforming industries by automating tasks and providing insights. Machine learning enables computers to learn from data.", "AI transforms industries through automation and insights via machine learning.", "summarization"),
                ("Space exploration missions have led to scientific discoveries and technological innovations. These advances benefit life on Earth.", "Space missions lead to scientific discoveries and beneficial technologies.", "summarization"),
                ("Renewable energy sources like solar and wind power are becoming more efficient and cost-effective. This transition is crucial for environmental sustainability.", "Renewable energy is becoming efficient and cost-effective for sustainability.", "summarization"),
                ("Digital education platforms are transforming how students learn and teachers deliver content. Online learning has become mainstream.", "Digital platforms transform education through online learning becoming mainstream.", "summarization"),
                ("Medical research has advanced significantly with new treatments and diagnostic tools. Personalized medicine is becoming a reality.", "Medical research advances with new treatments and personalized medicine reality.", "summarization"),
            ],
            'classification': [
                ("Scientists discover new species in deep ocean using advanced underwater robots.", "science", "classification"),
                ("Championship game ends with dramatic overtime victory in final minutes.", "sports", "classification"),
                ("Tech stocks surge after quantum computing breakthrough announcement.", "technology", "classification"),
                ("Central bank changes interest rates to combat rising inflation.", "business", "classification"),
                ("International leaders meet to discuss global trade policies.", "politics", "classification"),
                ("Archaeological findings reveal ancient mathematical knowledge.", "science", "classification"),
                ("Startup secures funding for sustainable energy solutions.", "technology", "classification"),
                ("Market analysts predict commodity price changes.", "business", "classification"),
                ("New vaccine shows promising results in clinical trials.", "science", "classification"),
                ("Athletes break world records at international competition.", "sports", "classification"),
            ],
            'general': [
                ("The future of artificial intelligence", "Artificial intelligence will continue to advance and transform various industries through machine learning, automation, and data analysis.", "general"),
                ("Space exploration and discovery", "Space missions help us understand the universe, develop new technologies, and potentially find life beyond Earth.", "general"),
                ("Renewable energy solutions", "Solar, wind, hydroelectric, and other renewable sources are crucial for sustainable energy and environmental protection.", "general"),
                ("Medical technology advances", "Modern medical technology includes robotic surgery, personalized medicine, and advanced diagnostic tools.", "general"),
                ("Global education systems", "Education systems worldwide are adapting to digital learning, skills-based training, and lifelong learning models.", "general"),
                ("Climate change mitigation", "Addressing climate change requires international cooperation, emission reductions, and sustainable technology development.", "general"),
                ("Digital transformation trends", "Digital technologies are reshaping business processes, customer experiences, and operational efficiency across industries.", "general"),
                ("Sustainable urban development", "Smart cities integrate technology, green infrastructure, and sustainable practices to improve quality of life.", "general"),
            ]
        }

        # Create validation sets
        self.validation_datasets = {
            task: data[:3] for task, data in self.datasets.items()
        }

        for task, data in self.datasets.items():
            logger.info(f"Added {len(data)} comprehensive fallback samples for {task}")

class EnhancedCEMOptimizer:
    def __init__(self, config: EnhancedConfig):
        self.config = config
        self.population_size = config.cem_population_size
        self.elite_ratio = config.cem_elite_ratio
        self.n_elite = max(1, int(self.population_size * self.elite_ratio))
        self.noise_std = config.cem_noise_std
        self.momentum = config.cem_momentum

    def optimize_adaptation(self, model, input_batch, target_batch,
                          adaptation_dim: int, max_steps: int = None):
        if max_steps is None:
            max_steps = self.config.cem_adaptation_steps

        # Better initialization
        population_mean = torch.zeros(adaptation_dim, device=device)
        population_std = torch.ones(adaptation_dim, device=device) * self.noise_std

        best_params = None
        best_score = float('-inf')
        convergence_history = []

        # Adaptive step size with better decay
        step_size = 1.0
        patience = 5
        no_improve_count = 0

        with timer("CEM Optimization"):
            for step in range(max_steps):
                try:
                    # Generate population with better sampling
                    population = torch.randn(self.population_size, adaptation_dim, device=device)
                    population = population * population_std * step_size + population_mean

                    # Softer clipping for better exploration
                    population = torch.clamp(population, -2.0, 2.0)

                    # Evaluate population
                    scores = self._batch_evaluate_adaptation_params(
                        model, input_batch, target_batch, population
                    )

                    # Handle invalid scores
                    valid_mask = torch.isfinite(scores) & (scores > -100)
                    if not valid_mask.any():
                        logger.warning(f"All CEM scores invalid at step {step}")
                        # Use random scores but continue
                        scores = torch.randn_like(scores) * 0.1 - 5.0
                        valid_mask = torch.ones_like(scores, dtype=torch.bool)

                    valid_scores = scores[valid_mask]
                    valid_population = population[valid_mask]

                    # Update best solution
                    if len(valid_scores) > 0:
                        current_best_idx = torch.argmax(valid_scores)
                        current_best_score = valid_scores[current_best_idx].item()

                        if current_best_score > best_score:
                            best_score = current_best_score
                            best_params = valid_population[current_best_idx].clone()
                            no_improve_count = 0
                        else:
                            no_improve_count += 1

                    # Update distribution with elite samples
                    n_elite_actual = min(self.n_elite, len(valid_scores))
                    if n_elite_actual > 0:
                        elite_indices = torch.topk(valid_scores, n_elite_actual)[1]
                        elite_samples = valid_population[elite_indices]

                        # Update mean and std
                        new_mean = elite_samples.mean(dim=0)
                        new_std = elite_samples.std(dim=0) + 1e-6

                        # Apply momentum
                        population_mean = self.momentum * population_mean + (1 - self.momentum) * new_mean
                        population_std = self.momentum * population_std + (1 - self.momentum) * new_std

                        # Clamp std to reasonable range
                        population_std = torch.clamp(population_std, 0.05, 1.0)

                        # Check convergence
                        mean_change = torch.norm(new_mean - population_mean).item()
                        convergence_history.append(mean_change)

                        # Adaptive step size
                        if no_improve_count >= patience:
                            step_size *= 0.8
                            no_improve_count = 0

                        if mean_change < self.config.cem_convergence_threshold:
                            logger.info(f"CEM converged at step {step} with score {best_score:.4f}")
                            break

                except Exception as e:
                    logger.error(f"CEM step {step} failed: {str(e)}")
                    continue

        return best_params, best_score, convergence_history

    def _batch_evaluate_adaptation_params(self, model, input_batch, target_batch, population):
        scores = torch.full((len(population),), float('-inf'), device=device)

        with torch.no_grad():
            for i, params in enumerate(population):
                try:
                    model.apply_adaptation_params(params)
                    outputs = model.forward_with_adaptation(
                        input_batch["input_ids"],
                        attention_mask=input_batch["attention_mask"]
                    )

                    if target_batch is not None:
                        loss = F.cross_entropy(
                            outputs.logits.view(-1, outputs.logits.size(-1)),
                            target_batch.view(-1),
                            ignore_index=-100,
                            reduction='mean'
                        )
                    else:
                        shift_logits = outputs.logits[..., :-1, :].contiguous()
                        shift_labels = input_batch["input_ids"][..., 1:].contiguous()
                        loss = F.cross_entropy(
                            shift_logits.view(-1, shift_logits.size(-1)),
                            shift_labels.view(-1),
                            ignore_index=-100,
                            reduction='mean'
                        )

                    score = -loss.item()
                    scores[i] = score if torch.isfinite(torch.tensor(score)) else -10.0

                except Exception as e:
                    logger.error(f"Evaluation of params {i} failed: {str(e)}")
                    scores[i] = -10.0

        return scores

@dataclass
class Episode:
    input_ids: torch.Tensor
    attention_mask: torch.Tensor
    generated_tokens: torch.Tensor
    log_probs: torch.Tensor
    rewards: torch.Tensor
    values: torch.Tensor
    task_type: str
    baseline_reward: float = 0.0
    is_finished: bool = True

class EnhancedSelfAdaptiveGPT2(nn.Module):
    def __init__(self, config: EnhancedConfig):
        super().__init__()
        self.config = config
        logger.info(f"Loading {config.model_name} on {device}")

        self.base_model = GPT2LMHeadModel.from_pretrained(config.model_name).to(device)
        if config.mixed_precision:
            self.base_model = self.base_model.float()

        self.base_model.gradient_checkpointing_enable()

        self.svd_components = {}
        self.adaptation_params = nn.ParameterDict()
        self.value_network = EnhancedValueNetwork(self.base_model.config.hidden_size)

        self.task_classifier = nn.Sequential(
            nn.Linear(self.base_model.config.hidden_size, config.num_experts),
            nn.Softmax(dim=-1)
        ).to(device)

        self.cem_optimizer = EnhancedCEMOptimizer(config)

        self._initialize_svd_decomposition()

        self.current_adaptation = None
        self.adaptation_history = deque(maxlen=50)

        if config.mixed_precision:
            self.scaler = torch.cuda.amp.GradScaler()

        logger.info(f"Model initialized with {self._count_parameters():,} parameters")

    def _count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def _initialize_svd_decomposition(self):
        logger.info("Initializing enhanced SVD decomposition")

        target_patterns = [
            'transformer.h.[0-3].mlp.c_fc',
            'transformer.h.[0-3].mlp.c_proj',
            'transformer.h.[0-3].attn.c_attn'
        ]

        target_patterns = [
            pattern.replace('[0-3]', str(i)) for i in range(4) for pattern in target_patterns
        ]

        decomposed_layers = 0
        for name, module in self.base_model.named_modules():
            if any(pattern in name for pattern in target_patterns) and hasattr(module, 'weight'):
                try:
                    weight = module.weight.data.to(device).float()
                    U, S, V = StabilizedSVDDecomposer.decompose_weight(
                        weight, self.config.svd_rank_ratio, self.config.svd_min_singular_value
                    )

                    if U is None:
                        logger.warning(f"SVD failed for {name}")
                        continue

                    self.svd_components[name] = {
                        'U': U.float(), 'S': S.float(), 'V': V.float(), 'original_shape': weight.shape
                    }

                    param_name = name.replace('.', '_')
                    adaptation_dim = len(S)
                    self.adaptation_params[param_name] = nn.Parameter(
                        torch.zeros(adaptation_dim, device=device, dtype=torch.float32),
                        requires_grad=True
                    )
                    decomposed_layers += 1

                except Exception as e:
                    logger.error(f"SVD failed for {name}: {str(e)}")
                    continue

        logger.info(f"SVD decomposition completed for {decomposed_layers} layers")
        logger.info(f"Total adaptation parameters: {sum(p.numel() for p in self.adaptation_params.values()):,}")

    def apply_adaptation_params(self, global_params: torch.Tensor):
        try:
            global_params = global_params.to(device).float()
            param_idx = 0

            for name in sorted(self.svd_components.keys()):
                param_name = name.replace('.', '_')
                if param_name in self.adaptation_params:
                    param_size = self.adaptation_params[param_name].size(0)
                    if param_idx + param_size <= len(global_params):
                        self.adaptation_params[param_name].data = global_params[param_idx:param_idx + param_size].float()
                        param_idx += param_size
                    else:
                        logger.warning(f"Insufficient parameters for {param_name}")
                        break
        except Exception as e:
            logger.error(f"Error applying adaptation params: {str(e)}")

    def get_total_adaptation_dim(self) -> int:
        return sum(param.size(0) for param in self.adaptation_params.values())

    def forward_with_adaptation(self, input_ids, attention_mask=None, use_adaptation=True):
        input_ids = input_ids.to(device)
        if attention_mask is not None:
            attention_mask = attention_mask.to(device)

        if not use_adaptation or not self.svd_components:
            with torch.cuda.amp.autocast(enabled=self.config.mixed_precision):
                return self.base_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=True
                )

        original_weights = {}
        try:
            # Apply adaptations
            for name, components in self.svd_components.items():
                module = dict(self.base_model.named_modules())[name]
                original_weights[name] = module.weight.data.clone()

                param_name = name.replace('.', '_')
                if param_name in self.adaptation_params:
                    adapted_weight = StabilizedSVDDecomposer.reconstruct_weight(
                        components['U'].float(), components['S'].float(), components['V'].float(),
                        self.adaptation_params[param_name].float()
                    )
                    if adapted_weight is not None:
                        module.weight.data = adapted_weight.to(module.weight.dtype)

            with torch.cuda.amp.autocast(enabled=self.config.mixed_precision):
                outputs = self.base_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=True
                )

            return outputs

        except Exception as e:
            logger.error(f"Forward with adaptation failed: {str(e)}")
            return None
        finally:
            # Restore original weights
            for name, original_weight in original_weights.items():
                dict(self.base_model.named_modules())[name].weight.data = original_weight

    def generate_episode(self, input_ids, attention_mask, max_new_tokens=50, task_type="general"):
        self.eval()
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        with torch.no_grad():
            try:
                with torch.cuda.amp.autocast(enabled=self.config.mixed_precision):
                    initial_outputs = self.forward_with_adaptation(input_ids, attention_mask)
                    if initial_outputs is None:
                        raise ValueError("Initial forward pass failed")
                    values = self.value_network(initial_outputs.hidden_states[-1])

                # Optimized generation configuration
                generation_config = {
                    "max_new_tokens": max_new_tokens,
                    "do_sample": True,
                    "temperature": self.config.temperature,
                    "top_p": self.config.top_p,
                    "top_k": 40,
                    "repetition_penalty": self.config.repetition_penalty,
                    "no_repeat_ngram_size": 3,
                    "pad_token_id": self.base_model.config.eos_token_id,
                    "eos_token_id": self.base_model.config.eos_token_id,
                    "return_dict_in_generate": True,
                    "output_scores": True
                }

                generated = self.base_model.generate(
                    input_ids, attention_mask=attention_mask, **generation_config
                )

                generated_tokens = generated.sequences[:, input_ids.size(1):]

                # Calculate log probabilities
                log_probs = []
                if hasattr(generated, 'scores') and generated.scores:
                    for i, score in enumerate(generated.scores):
                        if i < generated_tokens.size(1):
                            token_log_probs = F.log_softmax(score, dim=-1)
                            selected_log_probs = token_log_probs.gather(
                                1, generated_tokens[:, i:i+1]
                            )
                            log_probs.append(selected_log_probs.squeeze(-1))

                log_probs = torch.stack(log_probs, dim=1) if log_probs else torch.zeros(generated_tokens.size(), device=device)
                rewards = torch.ones(generated_tokens.size(), device=device) * 0.05

                return Episode(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    generated_tokens=generated_tokens,
                    log_probs=log_probs,
                    rewards=rewards,
                    values=values,
                    task_type=task_type
                )

            except Exception as e:
                logger.error(f"Episode generation failed: {str(e)}")
                dummy_tokens = torch.zeros((input_ids.size(0), 1), device=device, dtype=torch.long)
                return Episode(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    generated_tokens=dummy_tokens,
                    log_probs=torch.zeros_like(dummy_tokens, dtype=torch.float),
                    rewards=torch.zeros_like(dummy_tokens, dtype=torch.float),
                    values=torch.zeros(input_ids.size(0), device=device),
                    task_type=task_type
                )

    def compute_grpo_loss(self, episodes: List[Episode]):
        if not episodes:
            return torch.tensor(0.0, requires_grad=True, device=device)

        grouped_episodes = defaultdict(list)
        for episode in episodes:
            grouped_episodes[episode.task_type].append(episode)

        total_loss = torch.tensor(0.0, requires_grad=True, device=device)
        total_episodes = 0

        for task_type, task_episodes in grouped_episodes.items():
            if len(task_episodes) < 1:
                continue

            # Collect all rewards for normalization
            all_rewards = [ep.rewards.flatten() for ep in task_episodes if ep.rewards.numel() > 0]
            all_advantages = []

            if not all_rewards:
                continue

            all_rewards = torch.cat(all_rewards)
            if len(all_rewards) > 1 and self.config.grpo_reward_normalization:
                reward_mean = all_rewards.mean()
                reward_std = torch.clamp(all_rewards.std() + 1e-6, min=0.1, max=10.0)
            else:
                reward_mean = 0.0
                reward_std = 1.0

            task_loss = torch.tensor(0.0, requires_grad=True, device=device)
            valid_episodes = 0

            for episode in task_episodes:
                try:
                    if episode.rewards.numel() == 0 or episode.log_probs.numel() == 0:
                        continue

                    episode.values = episode.values.to(device)
                    episode.log_probs = episode.log_probs.to(device)
                    episode.rewards = episode.rewards.to(device)

                    # Normalize rewards
                    normalized_rewards = (episode.rewards - reward_mean) / reward_std if self.config.grpo_reward_normalization else episode.rewards
                    normalized_rewards = torch.clamp(normalized_rewards, -self.config.clip_rewards, self.config.clip_rewards)
                    normalized_rewards = normalized_rewards * self.config.reward_scaling

                    # Recompute values if needed
                    if not episode.values.requires_grad:
                        with torch.cuda.amp.autocast(enabled=self.config.mixed_precision):
                            outputs = self.forward_with_adaptation(episode.input_ids, episode.attention_mask)
                            if outputs is not None:
                                episode.values = self.value_network(outputs.hidden_states[-1])

                    # Handle dimension mismatch
                    if normalized_rewards.dim() == 2 and episode.values.dim() == 1:
                        values_expanded = episode.values.unsqueeze(1).expand_as(normalized_rewards)
                    elif normalized_rewards.dim() == 1 and episode.values.dim() == 1:
                        if len(normalized_rewards) > len(episode.values):
                            normalized_rewards = normalized_rewards.mean()
                            values_expanded = episode.values.mean()
                        else:
                            values_expanded = episode.values[:len(normalized_rewards)]
                    else:
                        values_expanded = episode.values

                    # Calculate advantages
                    advantages = normalized_rewards - values_expanded.detach()

                    # Advantage normalization
                    if advantages.numel() > 1:
                        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-6)

                    all_advantages.append(advantages)

                    # Policy loss with clipping
                    policy_loss = -(episode.log_probs * advantages).mean()
                    policy_loss = torch.clamp(policy_loss, -2.0, 2.0)

                    # Value loss
                    value_loss = F.mse_loss(values_expanded, normalized_rewards)

                    # Entropy loss
                    entropy_loss = -episode.log_probs.mean()

                    # Combined loss
                    episode_loss = (
                        policy_loss +
                        self.config.grpo_value_loss_coeff * value_loss +
                        self.config.grpo_entropy_coeff * entropy_loss
                    )

                    if torch.isfinite(episode_loss):
                        task_loss = task_loss + episode_loss
                        valid_episodes += 1

                except Exception as e:
                    logger.error(f"Episode loss computation failed: {str(e)}")
                    continue

            if valid_episodes > 0:
                total_loss = total_loss + task_loss / valid_episodes
                total_episodes += valid_episodes

        if all_advantages:
            self._log_advantage_stats(all_advantages)

        return total_loss / len(grouped_episodes) if total_episodes > 0 else torch.tensor(0.0, requires_grad=True, device=device)

    def _log_advantage_stats(self, advantages):
        advantages = torch.cat([a.flatten() for a in advantages if a.numel() > 0])
        if advantages.numel() > 0:
            logger.info(f"Advantage Stats - Mean: {advantages.mean().item():.4f}, Std: {advantages.std().item():.4f}, "
                        f"Min: {advantages.min().item():.4f}, Max: {advantages.max().item():.4f}")

    def adapt_for_inference(self, input_batch, target_batch=None):
        logger.info("Performing CEM adaptation")
        try:
            adaptation_dim = self.get_total_adaptation_dim()
            if adaptation_dim == 0:
                logger.warning("No adaptation parameters available")
                return 0.0, []

            # Ensure tensors are on correct device
            for key in input_batch:
                if isinstance(input_batch[key], torch.Tensor):
                    input_batch[key] = input_batch[key].to(device)

            if target_batch is not None:
                target_batch = target_batch.to(device)

            best_params, best_score, history = self.cem_optimizer.optimize_adaptation(
                self, input_batch, target_batch, adaptation_dim
            )

            if best_params is not None:
                self.apply_adaptation_params(best_params)
                self.current_adaptation = best_params.clone()
                self.adaptation_history.append({
                    'params': best_params.detach().cpu().numpy(),
                    'score': best_score,
                    'convergence_history': history
                })
                logger.info(f"CEM adaptation completed. Score: {best_score:.4f}")
            else:
                logger.warning("CEM adaptation failed")
                best_score = -10.0
                history = []

            return best_score, history

        except Exception as e:
            logger.error(f"CEM adaptation error: {str(e)}")
            return -10.0, []

class EnhancedGRPOTrainer:
    def __init__(self, config: EnhancedConfig):
        self.config = config
        self.tokenizer = GPT2Tokenizer.from_pretrained(config.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        logger.info("Initializing enhanced model")
        self.model = EnhancedSelfAdaptiveGPT2(config)

        # Optimizers
        adaptation_params = list(self.model.adaptation_params.values())
        other_params = list(self.model.task_classifier.parameters())

        self.policy_optimizer = torch.optim.AdamW(
            adaptation_params + other_params,
            lr=config.learning_rate,
            weight_decay=config.weight_decay,
            betas=(0.9, 0.999),
            eps=1e-8
        )

        self.value_optimizer = torch.optim.AdamW(
            self.model.value_network.parameters(),
            lr=config.learning_rate * 1.5,
            weight_decay=config.weight_decay,
            betas=(0.9, 0.999),
            eps=1e-8
        )

        # Schedulers
        total_steps = config.num_epochs * 100
        self.policy_scheduler = get_cosine_schedule_with_warmup(
            self.policy_optimizer,
            num_warmup_steps=config.warmup_steps,
            num_training_steps=total_steps
        )

        self.value_scheduler = get_cosine_schedule_with_warmup(
            self.value_optimizer,
            num_warmup_steps=config.warmup_steps,
            num_training_steps=total_steps
        )

        # Metrics tracking
        self.training_metrics = {
            "policy_loss": [], "value_loss": [], "entropy": [], "adaptation_magnitude": [],
            "cem_scores": [], "task_rewards": defaultdict(list), "gpu_memory_usage": [],
            "training_speed": [], "learning_rates": [], "gradient_norms": [], "episode_lengths": [],
            "policy_ratios": [], "advantage_distributions": [], "cem_convergence": [], "dataset_stats": {}
        }

        # Dataset loading
        self.dataset_loader = RobustDatasetLoader(config)
        self.datasets = self.dataset_loader.load_all_datasets()

        # Store dataset statistics
        self.training_metrics["dataset_stats"] = {
            "successful_downloads": self.dataset_loader.successful_downloads,
            "failed_downloads": self.dataset_loader.failed_downloads,
            "total_samples": sum(len(data) for data in self.datasets.values()),
            "task_distribution": {task: len(data) for task, data in self.datasets.items()}
        }

        # Reward function
        try:
            self.reward_function = ImprovedTaskRewardFunction()
        except Exception as e:
            logger.warning(f"Could not initialize reward function: {str(e)}")
            self.reward_function = ImprovedTaskRewardFunction()

        # Create output directory
        os.makedirs(config.output_dir, exist_ok=True)

        logger.info("Trainer initialized successfully")

    def create_optimized_dataloader(self, data, batch_size, is_validation=False):
        from torch.utils.data import Dataset, DataLoader

        class OptimizedDataset(Dataset):
            def __init__(self, data, tokenizer, max_length):
                # Filter out invalid data items upfront
                self.data = []
                for item in data:
                    if isinstance(item, (list, tuple)) and len(item) >= 3:
                        input_text, target_text, task_type = item[0], item[1], item[2]
                        if isinstance(input_text, str) and isinstance(target_text, str) and isinstance(task_type, str):
                            if len(input_text.strip()) > 0 and len(target_text.strip()) > 0:
                                self.data.append((input_text, target_text, task_type))

                self.tokenizer = tokenizer
                self.max_length = max_length
                logger.info(f"Dataset created with {len(self.data)} valid items from {len(data)} total items")

            def __len__(self):
                return len(self.data)

            def __getitem__(self, idx):
                try:
                    if idx >= len(self.data):
                        logger.error(f"Index {idx} out of range for dataset size {len(self.data)}")
                        return None

                    input_text, target_text, task_type = self.data[idx]

                    # Ensure strings are not empty
                    if not input_text or not target_text:
                        return None

                    inputs = self.tokenizer(
                        str(input_text), return_tensors="pt", truncation=True, max_length=self.max_length,
                        padding="max_length", add_special_tokens=True
                    )

                    targets = self.tokenizer(
                        str(target_text), return_tensors="pt", truncation=True, max_length=self.max_length // 2,
                        padding="max_length", add_special_tokens=True
                    )

                    return {
                        'input_ids': inputs['input_ids'].squeeze(0),
                        'attention_mask': inputs['attention_mask'].squeeze(0),
                        'target_ids': targets['input_ids'].squeeze(0),
                        'task_type': str(task_type),
                        'input_text': str(input_text)[:100],
                        'target_text': str(target_text)[:100]
                    }

                except Exception as e:
                    logger.error(f"Dataset item {idx} processing failed: {str(e)}")
                    return None

        dataset = OptimizedDataset(data, self.tokenizer, self.config.max_length)

        # Pre-filter None items to avoid issues in DataLoader
        valid_items = []
        for i in range(len(dataset)):
            item = dataset[i]
            if item is not None:
                valid_items.append(item)

        if not valid_items:
            logger.error("No valid dataset items found!")
            return None

        logger.info(f"Created DataLoader with {len(valid_items)} valid items")

        return DataLoader(
            valid_items,
            batch_size=batch_size,
            shuffle=not is_validation,
            num_workers=0,  # Avoid multiprocessing issues
            pin_memory=True,
            drop_last=True,
            collate_fn=self._collate_fn
        )

    def _collate_fn(self, batch):
        batch = [item for item in batch if item is not None]
        if not batch:
            return None

        collated = {}
        for key in batch[0].keys():
            if key in ['input_ids', 'attention_mask', 'target_ids']:
                collated[key] = torch.stack([item[key] for item in batch])
            else:
                collated[key] = [item[key] for item in batch]

        return collated

    def validate(self, val_dataloader):
        logger.info("Running validation")
        self.model.eval()
        val_metrics = {'loss': 0.0, 'rewards': defaultdict(list), 'episodes': 0}

        with torch.no_grad():
            for batch in val_dataloader:
                if batch is None:
                    continue

                try:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    task_types = batch['task_type']

                    episodes = []
                    for i in range(len(input_ids)):
                        episode = self.model.generate_episode(
                            input_ids[i:i+1].contiguous(),
                            attention_mask[i:i+1].contiguous(),
                            max_new_tokens=32,
                            task_type=task_types[i]
                        )

                        if episode.generated_tokens.numel() > 0:
                            generated_text = self.tokenizer.decode(
                                episode.generated_tokens[0], skip_special_tokens=True
                            )
                            target_text = batch['target_text'][i]
                            reward = self.reward_function.compute_reward(
                                generated_text, target_text, task_types[i]
                            )
                            val_metrics['rewards'][task_types[i]].append(reward)
                            episodes.append(episode)

                    if episodes:
                        loss = self.model.compute_grpo_loss(episodes)
                        if torch.isfinite(loss):
                            val_metrics['loss'] += loss.item()
                            val_metrics['episodes'] += len(episodes)

                except Exception as e:
                    logger.error(f"Validation batch failed: {str(e)}")
                    continue

        return val_metrics

    def train_enhanced_grpo(self):
        logger.info("Starting enhanced GRPO training")

        # Initialize wandb if configured
        if self.config.wandb_project:
            try:
                wandb.init(
                    project=self.config.wandb_project,
                    name=f"enhanced-grpo-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
                    config=self.config.__dict__
                )

                # Log dataset statistics
                wandb.log(self.training_metrics["dataset_stats"])

            except Exception as e:
                logger.warning(f"Wandb initialization failed: {str(e)}")

        # Prepare data
        all_data = []
        all_val_data = []

        for task_data in self.datasets.values():
            all_data.extend(task_data)

        for task_data in self.dataset_loader.validation_datasets.values():
            all_val_data.extend(task_data)

        if not all_data:
            logger.error("No training data available")
            return

        logger.info(f"Total training samples: {len(all_data)}")
        logger.info(f"Total validation samples: {len(all_val_data)}")

        # Create dataloaders
        dataloader = self.create_optimized_dataloader(all_data, self.config.batch_size)
        if dataloader is None:
            logger.error("Failed to create training dataloader")
            return

        val_dataloader = self.create_optimized_dataloader(all_val_data, self.config.batch_size, is_validation=True)
        if val_dataloader is None:
            logger.warning("Failed to create validation dataloader, continuing without validation")

        # Training loop
        self.model.train()
        global_step = 0

        for epoch in range(self.config.num_epochs):
            logger.info(f"Epoch {epoch + 1}/{self.config.num_epochs}")

            with timer(f"Epoch {epoch + 1}"):
                epoch_metrics = {'policy_loss': 0.0, 'episodes': 0, 'valid_episodes': 0}
                progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}")

                for batch_idx, batch in enumerate(progress_bar):
                    if batch is None:
                        logger.warning(f"Batch {batch_idx} is None, skipping")
                        continue

                    with timer(f"Batch {batch_idx}"):
                        try:
                            input_ids = batch['input_ids'].to(device)
                            attention_mask = batch['attention_mask'].to(device)
                            target_ids = batch['target_ids'].to(device)
                            task_types = batch['task_type']

                            # Generate episodes
                            episodes = []
                            for i in range(len(input_ids)):
                                try:
                                    episode = self.model.generate_episode(
                                        input_ids[i:i+1].contiguous(),
                                        attention_mask[i:i+1].contiguous(),
                                        max_new_tokens=32,
                                        task_type=task_types[i]
                                    )

                                    if episode.generated_tokens.numel() > 0:
                                        generated_text = self.tokenizer.decode(
                                            episode.generated_tokens[0], skip_special_tokens=True
                                        )
                                        target_text = batch['target_text'][i]

                                        # Compute reward
                                        reward = self.reward_function.compute_reward(
                                            generated_text, target_text, task_types[i]
                                        )

                                        episode.rewards.fill_(reward)
                                        episodes.append(episode)

                                        # Track metrics
                                        self.training_metrics["task_rewards"][task_types[i]].append(reward)
                                        self.training_metrics["episode_lengths"].append(episode.generated_tokens.size(1))

                                except Exception as e:
                                    logger.error(f"Episode generation failed for sample {i}: {str(e)}")
                                    continue

                            if not episodes:
                                logger.warning(f"No valid episodes generated for batch {batch_idx}")
                                continue

                            # Compute GRPO loss
                            grpo_loss = self.model.compute_grpo_loss(episodes)

                            if not torch.isfinite(grpo_loss) or not grpo_loss.requires_grad:
                                logger.warning("Invalid GRPO loss, skipping batch")
                                continue

                            # Backward pass
                            if self.config.mixed_precision:
                                scaled_loss = self.model.scaler.scale(grpo_loss / self.config.gradient_accumulation_steps)
                                scaled_loss.backward()
                            else:
                                (grpo_loss / self.config.gradient_accumulation_steps).backward()

                            # Optimizer step
                            if (batch_idx + 1) % self.config.gradient_accumulation_steps == 0:
                                if self.config.mixed_precision:
                                    self.model.scaler.unscale_(self.policy_optimizer)
                                    self.model.scaler.unscale_(self.value_optimizer)

                                    policy_grad_norm = torch.nn.utils.clip_grad_norm_(
                                        self.policy_optimizer.param_groups[0]['params'], self.config.max_grad_norm
                                    )
                                    value_grad_norm = torch.nn.utils.clip_grad_norm_(
                                        self.value_optimizer.param_groups[0]['params'], self.config.max_grad_norm
                                    )

                                    self.model.scaler.step(self.policy_optimizer)
                                    self.model.scaler.step(self.value_optimizer)
                                    self.model.scaler.update()
                                else:
                                    policy_grad_norm = torch.nn.utils.clip_grad_norm_(
                                        self.policy_optimizer.param_groups[0]['params'], self.config.max_grad_norm
                                    )
                                    value_grad_norm = torch.nn.utils.clip_grad_norm_(
                                        self.value_optimizer.param_groups[0]['params'], self.config.max_grad_norm
                                    )

                                    self.policy_optimizer.step()
                                    self.value_optimizer.step()

                                # Scheduler step
                                if self.config.adaptive_learning_rate:
                                    self.policy_scheduler.step()
                                    self.value_scheduler.step()

                                self.policy_optimizer.zero_grad()
                                self.value_optimizer.zero_grad()

                                # Track gradients
                                self.training_metrics["gradient_norms"].append({
                                    'policy': policy_grad_norm.item() if torch.isfinite(policy_grad_norm) else 0.0,
                                    'value': value_grad_norm.item() if torch.isfinite(value_grad_norm) else 0.0
                                })

                                global_step += 1

                            # Update metrics
                            epoch_metrics['policy_loss'] += grpo_loss.item()
                            epoch_metrics['episodes'] += len(episodes)
                            epoch_metrics['valid_episodes'] += len([e for e in episodes if e.rewards.sum() != 0])

                            # Track GPU memory
                            if torch.cuda.is_available():
                                self.training_metrics["gpu_memory_usage"].append(torch.cuda.memory_allocated() / 1e9)

                            # Track adaptation magnitude
                            if self.model.adaptation_params:
                                total_magnitude = sum(torch.norm(param).item() for param in self.model.adaptation_params.values())
                                self.training_metrics["adaptation_magnitude"].append(total_magnitude)

                            # Update progress bar
                            progress_bar.set_postfix({
                                'Loss': f'{grpo_loss.item():.4f}',
                                'Episodes': len(episodes),
                                'GPU_MB': f'{self.training_metrics["gpu_memory_usage"][-1]*1000:.0f}' if torch.cuda.is_available() else 'N/A'
                            })

                            # Logging
                            if batch_idx % self.config.log_interval == 0:
                                self._log_training_progress(epoch, batch_idx, grpo_loss.item(), global_step)

                            # Memory cleanup
                            if batch_idx % 20 == 0:
                                torch.cuda.empty_cache()
                                gc.collect()

                        except Exception as e:
                            logger.error(f"Batch {batch_idx} failed: {str(e)}")
                            continue

                # Epoch summary
                epoch_metrics['policy_loss'] /= max(len(dataloader), 1)
                self.training_metrics["policy_loss"].append(epoch_metrics['policy_loss'])
                self.training_metrics["learning_rates"].append({
                    'policy': self.policy_optimizer.param_groups[0]['lr'],
                    'value': self.value_optimizer.param_groups[0]['lr']
                })

                # Validation
                if val_dataloader is not None:
                    val_metrics = self.validate(val_dataloader)
                    val_loss = val_metrics['loss'] / max(val_metrics['episodes'], 1)
                else:
                    val_loss = 0.0

                logger.info(f"Epoch {epoch + 1} completed: Avg Policy Loss: {epoch_metrics['policy_loss']:.4f}, "
                            f"Episodes: {epoch_metrics['episodes']}, Valid Episodes: {epoch_metrics['valid_episodes']}, "
                            f"Validation Loss: {val_loss:.4f}")

                # Save checkpoint
                if (epoch + 1) % self.config.save_interval == 0:
                    self.save_checkpoint(epoch + 1, global_step)

        logger.info("Enhanced GRPO training completed")

        # Final cleanup
        torch.cuda.empty_cache()
        gc.collect()

    def _log_training_progress(self, epoch, batch_idx, loss, global_step):
        if self.config.wandb_project:
            try:
                log_dict = {
                    "train/policy_loss": loss,
                    "train/epoch": epoch,
                    "train/global_step": global_step,
                }

                if self.training_metrics["gpu_memory_usage"]:
                    log_dict["train/gpu_memory_gb"] = self.training_metrics["gpu_memory_usage"][-1]

                if self.training_metrics["adaptation_magnitude"]:
                    log_dict["train/adaptation_magnitude"] = self.training_metrics["adaptation_magnitude"][-1]

                if self.training_metrics["cem_convergence"]:
                    log_dict["train/cem_convergence"] = np.mean(self.training_metrics["cem_convergence"][-1])

                wandb.log(log_dict, step=global_step)

            except Exception as e:
                logger.warning(f"Wandb logging failed: {str(e)}")

    def test_cem_adaptation(self):
        logger.info("Testing CEM adaptation")

        test_cases = [
            ("What is the capital of France?", "qa"),
            ("This movie was absolutely amazing!", "sentiment"),
            ("Summarize: Climate change is a major global challenge affecting weather patterns...", "summarization"),
            ("Classify: Breaking news from the world of technology and innovation.", "classification"),
            ("The future of technology looks bright", "general")
        ]

        cem_results = []

        for input_text, task_type in test_cases:
            logger.info(f"Testing {task_type}: {input_text[:50]}...")

            try:
                inputs = self.tokenizer(
                    input_text, return_tensors="pt", truncation=True, max_length=128, padding=True
                )

                for key in inputs:
                    inputs[key] = inputs[key].to(device)

                with timer(f"CEM adaptation for {task_type}"):
                    score, history = self.model.adapt_for_inference(inputs)

                self.training_metrics["cem_convergence"].append(history)

                # Generate text with adaptation
                self.model.eval()
                with torch.no_grad(), torch.cuda.amp.autocast(enabled=self.config.mixed_precision):
                    generated = self.model.base_model.generate(
                        inputs["input_ids"],
                        attention_mask=inputs["attention_mask"],
                        max_length=inputs["input_ids"].size(1) + 40,
                        temperature=self.config.temperature,
                        do_sample=True,
                        top_p=self.config.top_p,
                        top_k=40,
                        repetition_penalty=self.config.repetition_penalty,
                        no_repeat_ngram_size=3,
                        pad_token_id=self.tokenizer.eos_token_id,
                        eos_token_id=self.tokenizer.eos_token_id
                    )

                    generated_text = self.tokenizer.decode(generated[0], skip_special_tokens=True)

                cem_results.append({
                    "input": input_text,
                    "task_type": task_type,
                    "generated": generated_text,
                    "cem_score": score,
                    "convergence_steps": len(history),
                    "adaptation_time": time.time() - (time.time() - 1)
                })

                self.training_metrics["cem_scores"].append(score)

                logger.info(f"Input: {input_text}")
                logger.info(f"Generated: {generated_text[len(input_text):].strip()}")
                logger.info(f"CEM Score: {score:.4f}")

            except Exception as e:
                logger.error(f"CEM adaptation failed for {task_type}: {str(e)}")
                continue

        return cem_results

    def save_checkpoint(self, epoch: int, global_step: int):
        checkpoint_path = os.path.join(
            self.config.output_dir, f"enhanced_checkpoint_epoch_{epoch}_step_{global_step}.pt"
        )

        try:
            checkpoint = {
                'epoch': epoch,
                'global_step': global_step,
                'model_state_dict': self.model.state_dict(),
                'policy_optimizer_state_dict': self.policy_optimizer.state_dict(),
                'value_optimizer_state_dict': self.value_optimizer.state_dict(),
                'policy_scheduler_state_dict': self.policy_scheduler.state_dict(),
                'value_scheduler_state_dict': self.value_scheduler.state_dict(),
                'training_metrics': self.training_metrics,
                'config': self.config
            }

            if self.config.mixed_precision:
                checkpoint['scaler_state_dict'] = self.model.scaler.state_dict()

            torch.save(checkpoint, checkpoint_path)
            logger.info(f"Checkpoint saved to {checkpoint_path}")

        except Exception as e:
            logger.error(f"Failed to save checkpoint: {str(e)}")

    def generate_enhanced_report(self):
        logger.info("Generating enhanced training report")

        # Create comprehensive visualization
        fig = plt.figure(figsize=(24, 32))
        gs = fig.add_gridspec(8, 4, hspace=0.4, wspace=0.3)
        plt.style.use('default')
        colors = plt.cm.Set2(np.linspace(0, 1, 10))

        # Policy Loss
        ax1 = fig.add_subplot(gs[0, 0:2])
        if self.training_metrics["policy_loss"]:
            epochs = range(1, len(self.training_metrics["policy_loss"]) + 1)
            ax1.plot(epochs, self.training_metrics["policy_loss"],
                    'b-', linewidth=2, marker='o', markersize=4)
            ax1.set_title("Policy Loss Over Epochs", fontsize=14, fontweight='bold')
            ax1.set_xlabel("Epoch")
            ax1.set_ylabel("Loss")
            ax1.grid(True, alpha=0.3)
            ax1.set_ylim(bottom=0)

        # Task Rewards
        ax2 = fig.add_subplot(gs[0, 2:4])
        if self.training_metrics["task_rewards"]:
            task_names = list(self.training_metrics["task_rewards"].keys())
            avg_rewards = [np.mean(rewards) for rewards in self.training_metrics["task_rewards"].values()]
            std_rewards = [np.std(rewards) for rewards in self.training_metrics["task_rewards"].values()]

            ax2.bar(task_names, avg_rewards, yerr=std_rewards,
                   capsize=5, alpha=0.7, color=colors[:len(task_names)])
            ax2.set_title("Average Rewards by Task", fontsize=14, fontweight='bold')
            ax2.set_ylabel("Reward")
            ax2.tick_params(axis='x', rotation=45)
            ax2.grid(True, alpha=0.3, axis='y')

        # CEM Convergence
        ax3 = fig.add_subplot(gs[1, 0:2])
        if self.training_metrics["cem_convergence"]:
            for i, conv in enumerate(self.training_metrics["cem_convergence"][-5:]):
                if conv:  # Check if convergence data exists
                    ax3.plot(conv, label=f'Test {i+1}', alpha=0.6)
            ax3.set_title("CEM Convergence Curves", fontsize=14, fontweight='bold')
            ax3.set_xlabel("Step")
            ax3.set_ylabel("Mean Change")
            ax3.legend()
            ax3.grid(True, alpha=0.3)

        # GPU Memory Usage
        ax4 = fig.add_subplot(gs[1, 2:4])
        if self.training_metrics["gpu_memory_usage"]:
            steps = range(len(self.training_metrics["gpu_memory_usage"]))
            ax4.plot(steps, self.training_metrics["gpu_memory_usage"],
                    'g-', linewidth=1, alpha=0.7)
            ax4.set_title("GPU Memory Usage", fontsize=14, fontweight='bold')
            ax4.set_xlabel("Training Step")
            ax4.set_ylabel("Memory (GB)")
            ax4.grid(True, alpha=0.3)

        # Dataset Statistics
        ax5 = fig.add_subplot(gs[2, 0:2])
        if self.training_metrics["dataset_stats"]["task_distribution"]:
            tasks = list(self.training_metrics["dataset_stats"]["task_distribution"].keys())
            counts = list(self.training_metrics["dataset_stats"]["task_distribution"].values())

            ax5.pie(counts, labels=tasks, autopct='%1.1f%%', startangle=90,
                   colors=colors[:len(tasks)])
            ax5.set_title("Dataset Distribution by Task", fontsize=14, fontweight='bold')

        # Learning Rate Schedule
        ax6 = fig.add_subplot(gs[2, 2:4])
        if self.training_metrics["learning_rates"]:
            epochs = range(len(self.training_metrics["learning_rates"]))
            policy_lrs = [lr_dict['policy'] for lr_dict in self.training_metrics["learning_rates"]]
            value_lrs = [lr_dict['value'] for lr_dict in self.training_metrics["learning_rates"]]

            ax6.plot(epochs, policy_lrs, 'b-', label='Policy LR', linewidth=2)
            ax6.plot(epochs, value_lrs, 'r-', label='Value LR', linewidth=2)
            ax6.set_title("Learning Rate Schedule", fontsize=14, fontweight='bold')
            ax6.set_xlabel("Epoch")
            ax6.set_ylabel("Learning Rate")
            ax6.legend()
            ax6.grid(True, alpha=0.3)
            ax6.set_yscale('log')

        # Gradient Norms
        ax7 = fig.add_subplot(gs[3, 0:2])
        if self.training_metrics["gradient_norms"]:
            steps = range(len(self.training_metrics["gradient_norms"]))
            policy_grads = [grad_dict['policy'] for grad_dict in self.training_metrics["gradient_norms"]]
            value_grads = [grad_dict['value'] for grad_dict in self.training_metrics["gradient_norms"]]

            ax7.plot(steps, policy_grads, 'b-', label='Policy Grad Norm', alpha=0.7)
            ax7.plot(steps, value_grads, 'r-', label='Value Grad Norm', alpha=0.7)
            ax7.set_title("Gradient Norms", fontsize=14, fontweight='bold')
            ax7.set_xlabel("Training Step")
            ax7.set_ylabel("Gradient Norm")
            ax7.legend()
            ax7.grid(True, alpha=0.3)

        # Episode Lengths
        ax8 = fig.add_subplot(gs[3, 2:4])
        if self.training_metrics["episode_lengths"]:
            ax8.hist(self.training_metrics["episode_lengths"], bins=20, alpha=0.7, color='purple')
            ax8.set_title("Episode Length Distribution", fontsize=14, fontweight='bold')
            ax8.set_xlabel("Episode Length")
            ax8.set_ylabel("Frequency")
            ax8.grid(True, alpha=0.3)

        plt.tight_layout()

        # Save the report
        report_path = os.path.join(self.config.output_dir, "enhanced_comprehensive_report.png")
        plt.savefig(report_path, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()

        logger.info(f"Enhanced report saved to {report_path}")

        # Generate detailed summary
        self._generate_detailed_summary()

        return report_path

    def _generate_detailed_summary(self):
        summary_path = os.path.join(self.config.output_dir, "enhanced_training_summary.txt")

        with open(summary_path, 'w') as f:
            f.write("=" * 100 + "\n")
            f.write("ENHANCED GPU-OPTIMIZED GRPO + CEM SELF-ADAPTIVE GPT2 TRAINING SUMMARY\n")
            f.write("=" * 100 + "\n\n")

            # System Configuration
            f.write("SYSTEM CONFIGURATION:\n")
            f.write("-" * 30 + "\n")
            f.write(f"Device: {device}\n")
            if torch.cuda.is_available():
                f.write(f"GPU: {torch.cuda.get_device_name()}\n")
                f.write(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB\n")
                f.write(f"CUDA Version: {torch.version.cuda}\n")
            f.write(f"PyTorch Version: {torch.__version__}\n")
            f.write(f"Mixed Precision: {self.config.mixed_precision}\n\n")

            # Model Configuration
            f.write("MODEL CONFIGURATION:\n")
            f.write("-" * 25 + "\n")
            f.write(f"Base Model: {self.config.model_name}\n")
            f.write(f"Max Sequence Length: {self.config.max_length}\n")
            f.write(f"Adaptation Rank: {self.config.adaptation_rank}\n")
            f.write(f"Number of Experts: {self.config.num_experts}\n")
            f.write(f"SVD Rank Ratio: {self.config.svd_rank_ratio}\n")

            total_params = self.model._count_parameters()
            adaptation_params = sum(p.numel() for p in self.model.adaptation_params.values())
            f.write(f"Total Parameters: {total_params:,}\n")
            f.write(f"Adaptation Parameters: {adaptation_params:,}\n")
            f.write(f"Parameter Efficiency: {adaptation_params/total_params*100:.3f}%\n\n")

            # Dataset Statistics
            f.write("DATASET STATISTICS:\n")
            f.write("-" * 25 + "\n")
            ds_stats = self.training_metrics["dataset_stats"]
            f.write(f"Successful HF Downloads: {ds_stats['successful_downloads']}\n")
            f.write(f"Failed HF Downloads: {ds_stats['failed_downloads']}\n")
            f.write(f"Total Training Samples: {ds_stats['total_samples']:,}\n")
            f.write("Task Distribution:\n")
            for task, count in ds_stats['task_distribution'].items():
                f.write(f"  - {task}: {count:,} samples\n")
            f.write("\n")

            # Training Results
            f.write("TRAINING RESULTS:\n")
            f.write("-" * 20 + "\n")
            if self.training_metrics["policy_loss"]:
                initial_loss = self.training_metrics["policy_loss"][0]
                final_loss = self.training_metrics["policy_loss"][-1]
                best_loss = min(self.training_metrics["policy_loss"])
                f.write(f"Initial Policy Loss: {initial_loss:.4f}\n")
                f.write(f"Final Policy Loss: {final_loss:.4f}\n")
                f.write(f"Best Policy Loss: {best_loss:.4f}\n")
                improvement = ((initial_loss - final_loss) / initial_loss * 100)
                f.write(f"Loss Improvement: {improvement:.2f}%\n")

            if self.training_metrics["cem_scores"]:
                f.write(f"Average CEM Score: {np.mean(self.training_metrics['cem_scores']):.4f}\n")
                f.write(f"Best CEM Score: {max(self.training_metrics['cem_scores']):.4f}\n")

            # Task-specific rewards
            f.write("\nTASK-SPECIFIC PERFORMANCE:\n")
            f.write("-" * 30 + "\n")
            for task, rewards in self.training_metrics["task_rewards"].items():
                if rewards:
                    f.write(f"{task.capitalize()}:\n")
                    f.write(f"  Average Reward: {np.mean(rewards):.4f}\n")
                    f.write(f"  Best Reward: {max(rewards):.4f}\n")
                    f.write(f"  Std Deviation: {np.std(rewards):.4f}\n")

            f.write("\n" + "=" * 100 + "\n")

        logger.info(f"Detailed summary saved to {summary_path}")

def run_comprehensive_evaluation(trainer: EnhancedGRPOTrainer):
    """Run comprehensive model evaluation"""
    logger.info("COMPREHENSIVE ENHANCED MODEL EVALUATION")

    # Test CEM adaptation
    cem_results = trainer.test_cem_adaptation()

    # Save evaluation results
    evaluation_path = os.path.join(trainer.config.output_dir, "enhanced_evaluation_results.json")
    evaluation_data = {
        "cem_results": cem_results,
        "dataset_statistics": trainer.training_metrics["dataset_stats"],
        "training_summary": {
            "total_epochs": trainer.config.num_epochs,
            "final_policy_loss": trainer.training_metrics["policy_loss"][-1] if trainer.training_metrics["policy_loss"] else None,
            "average_cem_score": np.mean(trainer.training_metrics["cem_scores"]) if trainer.training_metrics["cem_scores"] else None,
            "task_performance": {
                task: {
                    "avg_reward": np.mean(rewards),
                    "std_reward": np.std(rewards),
                    "max_reward": max(rewards),
                    "min_reward": min(rewards)
                } for task, rewards in trainer.training_metrics["task_rewards"].items() if rewards
            }
        },
        "timestamp": datetime.now().isoformat(),
        "config": trainer.config.__dict__
    }

    try:
        with open(evaluation_path, 'w') as f:
            json.dump(evaluation_data, f, indent=2, default=str)
        logger.info(f"Evaluation results saved to {evaluation_path}")
    except Exception as e:
        logger.error(f"Failed to save evaluation results: {str(e)}")

    return evaluation_data

def main():
    """Main execution function"""
    logger.info("Starting Enhanced GPU-Optimized GRPO + CEM Pipeline with Robust HuggingFace Integration")

    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
        torch.cuda.manual_seed_all(42)

    # Configuration
    config = EnhancedConfig()

    logger.info(f"Configuration Summary:")
    logger.info(f"  Model: {config.model_name}")
    logger.info(f"  Batch Size: {config.batch_size}")
    logger.info(f"  Learning Rate: {config.learning_rate}")
    logger.info(f"  Epochs: {config.num_epochs}")
    logger.info(f"  Max Length: {config.max_length}")
    logger.info(f"  Mixed Precision: {config.mixed_precision}")
    logger.info(f"  Use Fallback Data Only: {config.use_fallback_data_only}")
    logger.info(f"  Enable Internet Check: {config.enable_internet_check}")
    logger.info(f"  Output Directory: {config.output_dir}")

    try:
        # Initialize trainer
        logger.info("Initializing enhanced trainer with robust dataset loading...")
        trainer = EnhancedGRPOTrainer(config)

        # Check if we have sufficient data
        total_samples = sum(len(data) for data in trainer.datasets.values())
        if total_samples == 0:
            logger.error("No training data loaded, attempting fallback configuration")
            config.use_fallback_data_only = True
            trainer = EnhancedGRPOTrainer(config)
            total_samples = sum(len(data) for data in trainer.datasets.values())

            if total_samples == 0:
                logger.error("Still no data available! Please check your configuration.")
                return

        # Log dataset loading success
        logger.info(f"Successfully loaded {total_samples:,} training samples across {len(trainer.datasets)} tasks")
        logger.info(f"HuggingFace datasets loaded: {trainer.dataset_loader.successful_downloads}")
        logger.info(f"Failed downloads: {trainer.dataset_loader.failed_downloads}")

        # Start training
        logger.info("Starting enhanced GRPO training...")
        trainer.train_enhanced_grpo()

        # Run evaluation
        logger.info("Running comprehensive evaluation...")
        evaluation_results = run_comprehensive_evaluation(trainer)

        # Generate reports
        logger.info("Generating comprehensive reports...")
        report_path = trainer.generate_enhanced_report()

        # Final summary
        logger.info("\n" + "="*60)
        logger.info("🎉 ENHANCED PIPELINE COMPLETED SUCCESSFULLY! 🎉")
        logger.info("="*60)
        logger.info(f"📁 Results saved in: {config.output_dir}")
        logger.info(f"📊 Training report: {report_path}")

        # Performance summary
        if trainer.training_metrics["policy_loss"]:
            initial_loss = trainer.training_metrics["policy_loss"][0]
            final_loss = trainer.training_metrics["policy_loss"][-1]
            improvement = ((initial_loss - final_loss) / initial_loss * 100)
            logger.info(f"📈 Policy Loss Improvement: {improvement:.1f}%")

        if trainer.training_metrics["cem_scores"]:
            avg_cem_score = np.mean(trainer.training_metrics["cem_scores"])
            logger.info(f"🎯 Average CEM Score: {avg_cem_score:.3f}")

        # Dataset summary
        ds_stats = trainer.training_metrics["dataset_stats"]
        logger.info(f"📊 Dataset Summary:")
        logger.info(f"   • HuggingFace downloads: {ds_stats['successful_downloads']}/{ds_stats['successful_downloads'] + ds_stats['failed_downloads']}")
        logger.info(f"   • Total samples: {ds_stats['total_samples']:,}")
        logger.info(f"   • Tasks: {len(ds_stats['task_distribution'])}")

        # Next steps
        logger.info("\n🚀 Next Steps:")
        logger.info("  1. Review the comprehensive report for detailed analysis")
        logger.info("  2. Experiment with different hyperparameters")
        logger.info("  3. Test on additional datasets or custom data")
        logger.info("  4. Fine-tune for specific downstream tasks")
        logger.info("  5. Deploy the model for inference")

        # Troubleshooting tips
        logger.info("\n💡 Tips:")
        logger.info("  • If dataset loading fails, set use_fallback_data_only=True")
        logger.info("  • The fallback data is high-quality and sufficient for testing")
        logger.info("  • Check the detailed summary for performance metrics")
        logger.info("  • Use the saved checkpoints for inference or further training")

    except KeyboardInterrupt:
        logger.warning("Training interrupted by user")
    except Exception as e:
        logger.error(f"Pipeline failed with error: {str(e)}")
        traceback.print_exc()
    finally:
        # Cleanup
        logger.info("Performing cleanup...")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        # Close wandb if initialized
        if config.wandb_project:
            try:
                wandb.finish()
            except:
                pass

        logger.info("Cleanup completed. Pipeline finished.")

if __name__ == "__main__":
    main()

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ryanmacanzie (ryanmacanzie-shoolini-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1:   0%|          | 0/172 [00:00<?, ?it/s]

wandb: WARNING Tried to log to step 0 that is less than the current step 1. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
